该Python notebook利用langchain的QA chain，结合Chroma来实现PDF文档Analysis-and-Comparison-between-Optimism-and-StarkNet.pdf的语义化搜索。

该PDF文档共61页。通过本notebook，我们演示该字数规模的文件的语义化索引的OpenAI API开销。

使用时，在本地创建`.env`，并如`.env.example`所示，设置有效的OpenAI API Key即可。

In [6]:
%pip install openai > /dev/null
%pip install chromadb > /dev/null
%pip install langchain > /dev/null

In [66]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [4]:
# PDF_NAME='/content/drive/MyDrive/63_73_OCR_compressed.pdf'
PDF_NAME='data/63-73.pdf'

In [ ]:
# PyMuPDFLoader (X) 한글 안됨
# from langchain.document_loaders import PyMuPDFLoader

# def load_pdf():
#   return PyMuPDFLoader(PDF_NAME).load()

In [2]:
!pip install pypdf

In [5]:

from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_NAME)
docs = loader.load_and_split()
docs

[]

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install pymupdf

In [25]:
docs= load_pdf()

NameError: name 'load_pdf' is not defined

In [20]:
# f = open("/content/drive/MyDrive/news.txt", 'r')
f = open("data/news.txt", 'r')
docs = f.readlines()
f.close()

In [21]:
docs

['삼성전자 파운드리사업부 최고기술책임자(CTO) 자리에 또다시 변화가 생겼다. 지난 7월 선임된 CTO가 물러난 것으로 확인됐다\n',
 '\n',
 ' 28일 업계에 따르면 삼성전자 파운드리사업부 CTO를 맡던 부사장급 임원이 지난달 말 정기 인사에서 보좌역으로 전환됐다\n',
 '\n',
 ' 보좌역은 경영진 대상으로 사업을 자문하거나 지원하는 보직이다. CTO로 선임된 지 약 5개월 만에 생긴 이례적 변화다. 1년에 CTO가 두 번이나 바뀌게 됐다\n',
 '\n',
 ' 후임 CTO는 아직 결정되지 않았다. 삼성 연말 인사가 있은 지 한 달이 지났지만 파운드리 CTO는 공석으로 남아 있다. 필요 R&D는 파운드리사업부 내 기술개발 관련 임원들이 팀단위로 이끌고 있는 것으로 전해졌다.']

In [26]:
len(docs[0].page_content)
#len(docs[0])

AttributeError: 'str' object has no attribute 'page_content'

In [27]:
print (f'You have {len(docs)} document(s) in your data')
print (f'There are {len(docs[0].page_content)} characters in the first page of your document')

total = 0
for doc in docs:
  total += len(doc.page_content)
print (f'There are {total} characters in your document')

You have 7 document(s) in your data


AttributeError: 'str' object has no attribute 'page_content'

In [121]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

In [28]:
print (f'Now you have {len(split_docs)} documents')
#print (f'Now you have {len(docs)} documents')

NameError: name 'split_docs' is not defined

In [29]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('HOME')
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

ModuleNotFoundError: No module named 'google.colab'

In [30]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

NameError: name 'OPENAI_API_KEY' is not defined

In [125]:
# persist_directory = 'starknet'
# collection_name = 'starknet_index'
persist_directory = 'novel'
collection_name = 'novel_index'

In [126]:
from langchain.callbacks import get_openai_callback

In [127]:
with get_openai_callback() as cb:
    vectorstore = Chroma.from_documents(split_docs, embeddings, collection_name=collection_name, persist_directory=persist_directory)
    #vectorstore = Chroma.from_texts(docs, embeddings, collection_name=collection_name, persist_directory=persist_directory)
    vectorstore.persist()
    print(cb)


Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [128]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [129]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

chain = load_qa_chain(llm, chain_type="stuff")

# Load the vectorstore from disk
vectordb = Chroma(collection_name=collection_name, persist_directory=persist_directory, embedding_function=embeddings)

query = "장날에 무슨일이 일어났나요?"
#docs = vectorstore.similarity_search(query, 3, include_metadata=True)
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents(query)

[Document(page_content='지에 대한 기억들，마음속 사소한 궁금증을 터놓을 데가 없었다 \n이제는 영도와 부산 밖의 세상이 어떻게 돌아가는지 물어볼 사람 \n이 생겼다 한수는 선자가 하루를 어떻게 보냈는지 듣고 싶어 했 \n다 선자의 꿈이 무엇인지까지 알고 싶어 했다 가끔 선자가 난처 \n한 일을 처리하거나 사람을 상대하는 법을 모를 때면 해결책을 \n알려주었다 늘 한수는 문제를 해결할 묘안을 생각해냈다 두 사 \n람은 선자의 어머니 이야기는 절대 하지 않았다.\n장에서 일하고 있는 한수를 보면 낯설었다 선자와 함께 있을 \n때의 한수와는 다른 사람 같았다 한수는 선자의 친구이자 오빠 \n71', metadata={'page': 8, 'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf'}),
 Document(page_content='점 편해졌다 가을이 오자 바닷바람이 상쾌하고 쌀쌀해졌지만 선 \n자는 추운 줄도 몰랐다\n9월 초에 닷새 내내 비가 왔다 마침내 비가 개자 양진이 선자 \n에게 다음 날 아침에 태종대 숲에 가서 버섯을 따 오라고 시켰다 \n선자는 버섯 따는 것을 좋아했고，마침 바닷가에서 한수를 만날 \n참이라 맨날 하는 일이 아닌 색다른 일을 하러 간다고 말할 생각', metadata={'page': 9, 'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf'}),
 Document(page_content='었고 이는 선자가 미처 예상하지 못한 일이었다. 한수를 만나기 \n전에는 자기 생활에 대해 이야기할 사람이 없었다 하숙인들의 우 \n스운 습관들，어머니 밑에서 일하는 자매들과 나눈 대화들，아버 \n지에 대한 기억들，마음속 사소한 궁금증을 터놓을 데가 없었다 \n이제는 영도와 부산 밖의 세상이 어떻게 돌아가는지 물어볼 사람', metadata={'page': 8, 'source': '/content/drive/MyDrive/

In [98]:
query = "후임 CTO가 결정되었나?"
retriever.get_relevant_documents(query)

[Document(page_content='후임 CTO는 아직 결정되지 않았다. 삼성 연말 인사가 있은 지 한 달이 지났지만 파운드리 CTO는 공석으로 남아 있다. 필요 R&D는 파운드리사업부 내 기술개발 관련 임원들이 팀단위로 이끌고 있는 것으로 전해졌다.'),
 Document(page_content='보좌역은 경영진 대상으로 사업을 자문하거나 지원하는 보직이다. CTO로 선임된 지 약 5개월 만에 생긴 이례적 변화다. 1년에 CTO가 두 번이나 바뀌게 됐다.\n'),
 Document(page_content='삼성전자 파운드리사업부 최고기술책임자(CTO) 자리에 또다시 변화가 생겼다. 지난 7월 선임된 CTO가 물러난 것으로 확인됐다.\n'),
 Document(page_content='28일 업계에 따르면 삼성전자 파운드리사업부 CTO를 맡던 부사장급 임원이 지난달 말 정기 인사에서 보좌역으로 전환됐다.\n')]

In [130]:
print(chain.document_prompt)

input_variables=['page_content'] template='{page_content}'


In [131]:
for doc in docs:
    print(doc.metadata)

{'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf', 'page': 0}
{'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf', 'page': 1}
{'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf', 'page': 2}
{'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf', 'page': 3}
{'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf', 'page': 4}
{'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf', 'page': 5}
{'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf', 'page': 6}
{'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf', 'page': 7}
{'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf', 'page': 8}
{'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf', 'page': 9}
{'source': '/content/drive/MyDrive/63_73_OCR_compressed.pdf', 'page': 10}


In [132]:
print(chain.prompt_length(docs, question='What is starknet?'))

8021


In [140]:
query = "한수랑 선자는 둘 다 섬 출신인가요? 한국어로 대답해줘요."

with get_openai_callback() as cb:
    print(chain.run(input_documents=split_docs[:10], question=query))
    print(cb)

 네, 한수와 선자는 둘 다 섬 출신입니다.
Tokens Used: 1770
	Prompt Tokens: 1749
	Completion Tokens: 21
Successful Requests: 1
Total Cost (USD): $0.0026655000000000003
